# Here I'll just implement that :
http://dl.acm.org/citation.cfm?doid=2600428.2609514

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from ast import literal_eval
%matplotlib inline

In [2]:
df = pd.read_csv('data/train.csv')
df['recipient_id'] = df['recipient_id'].apply(literal_eval)
df = df.drop('Unnamed: 0', 1)
X, y = df[['sender_id', 'body']].values, df['recipient_id'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [3]:
df.head()

,sender,sender_id,mid,date,body,recipient_id,recipients
0,christian.yoder@enron.com,104,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,"[125, 126, 127, 117, 128, 129, 130, 131, 132]",robert.badeer@enron.com murray.o neil@enron.co...
1,heather.dunton@enron.com,113,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,"[50, 125, 126, 127, 133, 28, 117, 134, 135, 13...",kim.ward@enron.com robert.badeer@enron.com mur...
2,janel.guerrero@enron.com,49,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,"[125, 146, 147]",robert.badeer@enron.com john.massey@enron.com ...
3,tim.belden@enron.com,117,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,"[125, 130]",robert.badeer@enron.com jeff.richter@enron.com
4,christian.yoder@enron.com,104,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,"[148, 149, 125, 150]",pgillman@schiffhardin.com kamarlantes@calpx.co...


Total number of addresses is 9874.

### First, let's compute the social graph information we need

In [4]:
n_train = len(X_train)
n_senders = 125
n_people = 9874
graph = np.zeros((n_senders, n_people))
for i in range(n_train):
    sender = X_train[i][0]
    for recipient in y[i]:
        graph[sender][recipient] += 1
        
total_received = np.sum(graph, axis=0)
# if someone never received message, we put 1 instead of 0 to avoid arthmetical errors...
for i in range(len(total_received)):
    if total_received[i] == 0:
        total_received[i] = 1

graph_n = graph/total_received

In [5]:
graph

array([[ 0.,  1.,  2., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  2., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  2.,  0., ...,  0.,  0.,  0.]])

In [6]:
print(np.sum(graph == 0.), n_senders*n_people)

(1136561, 1234250)


The graph is really sparse (as one could expect), so changing the format would be a good idea if it was very large. But here operations are quite fast already.

### Now let's do the bag-of-word, tokenizing,... things

In [7]:
print(X_train[3][1])

MIDDLE EAST, AFRICA & ASIA-PACIFIC - US agency in threat on Indian loan guarantees.Financial Times, 09/10/01Manager s Journal: The Welch Legacy: Creative DestructionThe Wall Street Journal, 09/10/01RWE Unit Wants Top Spot Among Europe s Energy TradersDow Jones International News, 09/10/01INTERVIEW:Oman Delays LNG Deliveries To India Dabhol PwrDow Jones Energy Service, 09/10/01UK: Second phase of LME screen trade starts steady.Reuters English News Service, 09/10/01INDIA: US Exim may encash Enron India unit guarantee-paper.Reuters English News Service, 09/10/01INDIA PRESS: Dabhol Proj Cost May Rise $708M Due To DelayDow Jones International News, 09/10/01Dabhol impasse to cost MSEB Rs 15,000crThe Economic Times, 09/10/01Vajpayee Asks Indian Govt to Resolve Enron Power Plant DisputeBloomberg, 09/10/01MIDDLE EAST, AFRICA & ASIA-PACIFIC - US agency in threat on Indian loan guarantees.By KHOZEM MERCHANT.09/10/2001Financial Times(c) 2001 Financial Times Limited . All Rights ReservedThe US Expo

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
vect = CountVectorizer(min_df=3, binary=True)
X_body = vect.fit_transform(X_train[:,1])

In [10]:
from collections import defaultdict

n_words = X_body.shape[1]
fwrs = [defaultdict(lambda: defaultdict(lambda: 0.)) for i in range(n_words)]
fwr = [defaultdict(lambda: 0.) for i in range(n_words)]
fw = np.array(np.sum(X_body, axis=0))[0].astype(float)

In [11]:
import itertools

cx = X_body.tocoo()
for i,j in itertools.izip(cx.row, cx.col):
    for recipient_id in y_train[i]:
        fwrs[j][X_train[i][0]][recipient_id] += 1
        fwr[j][recipient_id] += 1

In [12]:
l, g, b = .6, .2, .2
terms = [defaultdict(lambda: defaultdict(lambda: 0)) for i in range(n_words)]
for w in range(n_words):
    for s,d in fwrs[w].iteritems():
        for r,v in d.iteritems():
            terms[w][s][r] = np.log(l*v/max(1., graph[s][r]) + g*fwr[w][r]/total_received[r] + b*fw[w]/n_train)

### Fit is done, let's predict on test

Predict on only a small part of the test because we don't know how to compute the probabilities fast enough for now.

In [13]:
n_small = 100
X_test_body = vect.transform(X_test[:n_small,1])

In [31]:
y_predict = np.zeros((n_small, 10), dtype=int)

In [15]:
recipient_lists = [set() for i in range(125)]
for i in range(125):
    for j in range(n_people):
        if graph[i][j] != 0:
            recipient_lists[i].add(j)

In [20]:
cxt = X_test_body.tocoo()
cur = 0
for i,j in itertools.izip(cxt.row, cxt.col):
    cur += len(recipient_lists[X_test[i][0]])
cur

17529678

In [19]:
cxt = X_test_body.tocoo()
probas = [defaultdict(lambda: 0) for i in range(n_small)]
cur = 0
for i,j in itertools.izip(cxt.row, cxt.col):
    s = X_test[i][0]
    for r in recipient_lists[s]:
        if(probas[i][r] == 0):
            probas[i][r] = np.log(graph_n[s][r]) + np.log(total_received[r]/n_train)
        probas[i][r] += terms[j][s][r]

In [107]:
len(recipient_lists[10])

455

In [22]:
import operator

In [32]:
for i in range(n_small):
    bests = sorted(probas[i].iteritems(), key=operator.itemgetter(1), reverse=True)[:10]
    for j in range(10):
        y_predict[i][j] = bests[j][0]

In [34]:
def ap(recommanded, real):
    real_set = set(real)
    cur = 0.
    n = len(recommanded)
    ans = 0.
    for k in range(1, n+1):
        if recommanded[k-1] in real_set:
            cur += 1
            ans += cur/k
    return ans/min(n, len(real))

def MAP(recommanded, real):
    ans = 0.
    for i in range(len(recommanded)):
        ans += ap(recommanded[i], real[i])
    return ans/len(recommanded)

MAP(y_predict, y_test)

0.1715183484504913

In [53]:
a = 14
print y_predict[a]
print y_test[a]
print ap(y_predict[a], y_test[a])

[1633 1066 1645 1343 2470 2291 2199  331  838 1348]
[1633]
1.0
